#Exercise 2

For this exercise, we make some simple visualizations of particles subject to different force functions.

We use the following packages:

NumPy -> allows us to use some math operations and NumPy arrays

Pandas -> gives us use of a DataFrame which will store experiment data in a 2D array used for the visualizations

Plotly -> allows us to make interactive data visualizations

Plotly.express -> allows us to create plots in a single line

Plotly.io -> controls how Plotly's visualizations are displayed

In [ ]:
import numpy as np

import pandas as pd

import plotly.express as px
import plotly.io as pio
from fontTools.misc.arrayTools import vectorLength

# Use an interactive frame for visualizations
pio.renderers.default = 'iframe'

Below we get some methods we will need to do the math and generate the figures out of the way. Read through this to understand the rest of the program.

In [ ]:
# You don't need to change anything in this block

def get_initial_followers_state(num_followers: int) -> tuple[np.ndarray, np.ndarray]:
    """
    Generates the initial position and velocity of the followers.
    :param num_followers: the number of followers
    :return: positions and velocities of the followers
    """
    # Create a np array with num_followers items, each with two values representing the x and y positions of the particle.
    # Each position [x, y] will obey x, y in range [-1, 1).
    followers_positions = 2 * (np.random.rand(num_followers, 2) - 0.5)
    # Create a np array with num_followers items, each with two values representing the x and y velocities of the particle.
    followers_velocities = np.zeros((num_followers, 2))
    return followers_positions, followers_velocities


def get_leader_position(t: int, leader_speed: float) -> np.ndarray:
    """
    Determines the position of the leader along the circle at time t.
    :param t: the time step
    :param leader_speed: the speed of the leader
    :return: the position of the leader
    """
    step_size = leader_speed
    distance_covered = step_size * t

    circle_radius = 1
    circle_circumference = 2 * circle_radius * np.pi
    percent_circle_completed = distance_covered / circle_circumference
    radians = percent_circle_completed * 2 * np.pi

    leader_x = np.cos(radians)
    leader_y = np.sin(radians)
    return np.array([leader_x, leader_y])


def magnitude(vector: np.ndarray) -> float:
    """
    :return: the magnitude of the provided vector
    """
    return float(np.linalg.norm(vector))


def distance_between(position: np.ndarray, goal: np.ndarray) -> float:
    """
    :param position: current position of the particle
    :param goal: goal position of the particle
    :return: distance between position and goal
    """
    return magnitude(goal - position)


def vector_towards(position, goal) -> np.ndarray:
    """
    :param position: current position of the particle
    :param goal: goal position of the particle
    :return: vector pointing from the position towards the goal
    """
    return goal - position


def to_dataframe(t: int, followers_positions: np.ndarray, leader_position: np.ndarray,
                 num_followers: int) -> pd.DataFrame:
    """
    Create a single DataFrame of the positions of all followers and the leader at time t.
    :param t: current time step
    :param followers_positions: the positions of the followers at time t
    :param leader_position: the position of the leader at time t
    :param num_followers: the number of followers
    :return: DataFrame containing the follower positions and leaders position at time t
    """
    df = pd.DataFrame()
    df['t'] = np.repeat(t, num_followers + 1)
    df['type'] = np.append(np.repeat('Follower', num_followers), 'Leader')
    df['x'] = np.append(followers_positions[:, 0], leader_position[0])
    df['y'] = np.append(followers_positions[:, 1], leader_position[1])
    df['particle_id'] = list(range(num_followers + 1))
    return df


def update(followers_positions: np.ndarray, followers_velocities: np.ndarray, leader_position: np.ndarray,
           inertia: float, force_function):
    """
    Calculates new positions and velocities based on the provided data.
    :param followers_positions: current position of the followers
    :param followers_velocities: current velocities of the followers
    :param leader_position: current position of the leader
    :param inertia: inertia variable
    :param force_function: the function used to calculate the force
    :return: new positions and velocities of the followers
    """
    new_velocities = inertia * followers_velocities + force_function(num_followers, followers_positions,
                                                                     leader_position)
    new_positions = followers_positions + new_velocities
    return new_positions, new_velocities


def run(leader_speed: float, num_followers: int, num_time_steps: int, inertia: float, force_function) -> pd.DataFrame:
    """
    Iterates over all time steps, updates the particle states, and writes each state to a dataframe
    :param leader_speed: the speed of the leader
    :param num_followers: the number of followers
    :param num_time_steps: the number of time steps
    :param inertia: inertia variable
    :param force_function: the function used to calculate the force
    :return: a DataFrame containing all the DataFrames from all the positions and velocities throughout the simulation
    """
    data = []
    leader_position = get_leader_position(0, leader_speed)
    followers_positions, followers_velocities = get_initial_followers_state(num_followers)
    data.append(to_dataframe(0, followers_positions, leader_position, num_followers))

    for t in range(1, num_time_steps, 1):
        leader_position = get_leader_position(t, leader_speed)
        followers_positions, followers_velocities = update(followers_positions, followers_velocities, leader_position,
                                                           inertia, force_function)
        data.append(to_dataframe(t, followers_positions, leader_position, num_followers))

    return pd.concat(data)

##Task A

Below I introduce two force functions. Some of the documentation is incomplete. Try completing it. It may be useful to modify the values of the parameters and run the simulations in proceeding code blocks.

In [ ]:
# These are two possible force functions (defining the attraction and repulsion forces experienced by the particles).
# You can play around with the parameters of the functions or create your own functions.

def force_random(num_followers: int, followers_positions: np.ndarray, leader_position: np.ndarray, d: float = 0.7,
                 k_followers: float = 0.01, k_leader: float = 0.5) -> np.ndarray:
    """
    Returns the force acting on each of the particles for one time step. Using linear attraction that is proportional to distance with a random offset.
    :param num_followers: the number of followers
    :param followers_positions: the positions of the followers
    :param leader_position: the position of the leader
    :param d: the upper bound of the distance between the particles, in terms of the strength of the force
    :param k_followers: the coefficient on the cohesion forces towards the followers
    :param k_leader: the coefficient on the force towards the leader
    :return: the force acting on each of the particles
    """
    force = np.zeros((num_followers, 2))

    for i in range(num_followers):
        vector_to_leader = vector_towards(followers_positions[i], leader_position)
        # Below is element-wise multiplication. For examples, [0.5, 2] * [2, 2] = [1, 4]
        force[i] += k_leader * (d - np.random.rand(2)) * vector_to_leader

        for j in range(num_followers):
            if i == j:
                continue
            vector_to_follower_j = vector_towards(followers_positions[i], followers_positions[j])
            force[i] += k_followers * (d - np.random.rand(2)) * vector_to_follower_j
    return force


def force_comfortable_distance(num_followers: int, followers_positions: np.ndarray, leader_position: np.ndarray,
                               d: float = 0.5, k_followers: float = 0.1, k_leader: float = 0.5):
    """
    Returns the force acting on each of the particles for one time step. Using comfortable distance between followers with each other and the leader.
    :param num_followers: the number of followers
    :param followers_positions: the positions of the followers
    :param leader_position: the position of the leader
    :param d: the comfortable distance between the particles
    :param k_followers: the coefficient on the cohesion forces towards the followers
    :param k_leader: the coefficient on the force towards the leader
    :return: the force acting on each of the particles
    """
    force = np.zeros((num_followers, 2))

    for i in range(num_followers):
        distance_to_leader = distance_between(followers_positions[i], leader_position)
        vector_to_leader = vector_towards(followers_positions[i], leader_position)
        leader_force = k_leader * (distance_to_leader - d) * vector_to_leader

        cohesion_force = 0
        for j in range(num_followers):
            if i == j:
                continue
            distance_to_follower_j = distance_between(followers_positions[i], followers_positions[j])
            vector_to_follower_j = vector_towards(followers_positions[i], followers_positions[j])
            cohesion_force += k_followers * (distance_to_follower_j - d) * vector_to_follower_j

        force[i] = leader_force + cohesion_force
    return force

Let's define some parameters to run the simulations.

In [ ]:
num_followers = 10
leader_speed = 0.1
num_time_steps = 100
inertia = 0.0
force_function = force_random

# Generate the dataframe for plotting
df = run(leader_speed, num_followers, num_time_steps, inertia, force_function)
# Display the first five rows of the dataframe
df.head()

##Task B

The following simulation will use the random force function. Before running the code try to predict how the swarm will behave.

1) Which shape does the swarm have (ring, blob, ...)?
2) Will the distances between particles and their closest neighbour be uniform or not?
3) Is the leader particle in the center of the swarm?
4) Are there any particles in front of the leader particle?
5) Is the center of the swarm inside, outside, or on the trajectory of the leader particle?

In [ ]:
force_function = force_random

df = run(leader_speed, num_followers, num_time_steps, inertia, force_function)

fig = px.scatter(df, x="x", y="y", color='type', symbol="type", animation_frame='t', animation_group='particle_id',
                 width=800, height=800)
fig.update_layout(xaxis_range=(-2.0, 2.0), yaxis_range=(-2.0, 2.0))
fig.show()

##Task C

1) Run the notebook without any changes and describe how the swarm behaves.

Solution: The swarm is following the leader, but behaves erratically and unstable

3) How does the behavior change if you don’t use the default force function, but instead the
provided simple comfortable distance function?

Solution: The swarm still follows the leader, but this time the movement is smooth and stable.

In [ ]:
force_function = force_comfortable_distance

df = run(leader_speed, num_followers, num_time_steps, inertia, force_function)

fig = px.scatter(df, x="x", y="y", color='type', symbol="type", animation_frame='t', animation_group='particle_id',
                 width=800, height=800)
fig.update_layout(xaxis_range=(-2.0, 2.0), yaxis_range=(-2.0, 2.0))
fig.show()

##Task D

1) In the two provided force functions, what does the parameter d control? What behavior
can you observe when using different values?

Solution: In case of the random force function, the parameter d controls the influence of the
randomness. In case of the simple comfortable distance function, it controls the comfortable
distance.

2) Can you add a new force function with implements function III for comfortable distance from the lecture? How does its behaviour differ from the provided comfortable distance function.

In [ ]:
# You have to modify this for the task

def force_function3(num_followers: int, followers_positions: np.ndarray, leader_position: np.ndarray, a: float = 0.05,
                    b: float = 0.2, c: float = 0.2) -> np.ndarray:
    force = np.zeros((num_followers, 2))

    for i in range(num_followers):
        distance_to_leader = distance_between(followers_positions[i], leader_position)
        vector_to_leader = vector_towards(followers_positions[i], leader_position)
        leader_force = (a - b * np.exp(-(distance_to_leader ** 2) / c)) * vector_to_leader

        cohesion_force = 0
        for j in range(num_followers):
            if i == j:
                continue
            distance_to_follower_j = distance_between(followers_positions[i], followers_positions[j])
            vector_to_follower_j = vector_towards(followers_positions[i], followers_positions[j])
            cohesion_force += (a - b * np.exp(-(distance_to_follower_j ** 2) / c)) * vector_to_follower_j

        force[i] = leader_force + cohesion_force
    return force


force_function = force_function3

df = run(leader_speed, num_followers, num_time_steps, inertia, force_function)
fig = px.scatter(df, x="x", y="y", color='type', symbol="type", animation_frame='t', animation_group='particle_id',
                 width=800, height=800)
fig.update_layout(xaxis_range=(-2.0, 2.0), yaxis_range=(-2.0, 2.0))
fig.show()